In [1]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import IB, Stock, Option
from datetime import datetime, timedelta
import pandas as pd

# Connection settings
IB_HOST = '127.0.0.1'
IB_PORT = 4002  # IB Gateway paper trading
CLIENT_ID = 33

In [2]:
ib = IB()
ib.connect(IB_HOST, IB_PORT, clientId=CLIENT_ID)
print(f'Connected: {ib.isConnected()}')

Connected: True


In [3]:
# Get all positions
positions = ib.positions()

# Filter to options only
option_positions = [p for p in positions if p.contract.secType == 'OPT' and p.position != 0 ]

print(f"Found {len(option_positions)} option positions:\n")
for p in option_positions:
    c = p.contract
    print(f"{c.symbol:6} {c.right} {c.strike:>7} {c.lastTradeDateOrContractMonth} qty={p.position:>3} avg={p.avgCost:.2f}")

Found 11 option positions:

JPM    P   322.5 20260116 qty=1.0 avg=589.05
NEOG   C     7.5 20260116 qty=-5.0 avg=115.30
NEOG   P     7.5 20260116 qty=5.0 avg=4.55
CNXC   C    40.0 20260116 qty=3.0 avg=310.70
DAL    P    71.0 20260116 qty=4.0 avg=240.70
WDFC   C   200.0 20260116 qty=10.0 avg=974.70
CNXC   P    40.0 20260116 qty=3.0 avg=275.70
DAL    C    71.0 20260116 qty=4.0 avg=235.70
JPM    C   322.5 20260116 qty=1.0 avg=596.05
BK     P   120.0 20260116 qty=4.0 avg=234.90
BK     C   120.0 20260116 qty=4.0 avg=181.14


In [26]:
# Close all option positions with market orders
# Each position is closed individually (not as combo) to avoid "riskless combination" rejection

orders_placed = []

for p in option_positions:
    contract = p.contract
    qty = abs(p.position)
    
    # Qualify the contract first
    ib.qualifyContracts(contract)
    
    # Create market order to close (SELL if long, BUY if short)
    action = 'SELL' if p.position > 0 else 'BUY'
    order = MarketOrder(action, qty)
    
    # Place the order
    trade = ib.placeOrder(contract, order)
    orders_placed.append((contract.symbol, contract.right, contract.strike, qty, trade))
    print(f"Placed {action} {qty}x {contract.symbol} {contract.right} {contract.strike}")
    
    # Small delay to avoid pacing violations
    ib.sleep(0.1)

print(f"\nPlaced {len(orders_placed)} orders")

Placed BUY 5.0x NEOG P 7.5

Placed 1 orders


In [27]:
# Wait for fills and check status
ib.sleep(2)

print("Order Status:\n")
for symbol, right, strike, qty, trade in orders_placed:
    status = trade.orderStatus.status
    filled = trade.orderStatus.filled
    avg_price = trade.orderStatus.avgFillPrice
    print(f"{symbol:6} {right} {strike:>7} qty={qty:>3} status={status:12} filled={filled} @ {avg_price:.2f}")

Order Status:

NEOG   P     7.5 qty=5.0 status=PreSubmitted filled=0.0 @ 0.00


In [28]:
# Verify positions are closed
ib.sleep(1)
remaining = [p for p in ib.positions() if p.contract.secType == 'OPT' and p.position != 0]
print(f"Remaining option positions: {len(remaining)}")
for p in remaining:
    c = p.contract
    print(f"  {c.symbol} {c.right} {c.strike} qty={p.position}")

Remaining option positions: 5
  NEOG P 7.5 qty=-5.0
  AEHR P 22.5 qty=5.0
  AEHR C 22.5 qty=5.0
  WDFC P 200.0 qty=10.0
  WDFC C 200.0 qty=10.0


In [33]:
# Disconnect
ib.disconnect()